In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
import parse_tools as pt

In [3]:
def iter_tables(tables,sleep=2):
    if isinstance(tables,dict):
        tables_list = tables.items()
    elif isinstance(tables,(list,tuple)):
        tables_list = enumerate(tables)
    tables_list = tqdm(tables_list)
    for table_id, table in tables_list:
        tables_list.set_description(table_id)
        display_html(table)
        time.sleep(sleep)
        clear_output()

def convert_time_to_minutes(time_str):
    if pd.notnull(time_str) and re.match(r'\d{1,2}:\d{2}',time_str):
        minutes, seconds = time_str.split(':')
        return float(minutes) + float(seconds) / 60.0
    else:
        return pd.NA

def convert_series_dtype(series):
    # backend = {'numpy':np, 'pyarrow':pa}[backend]
    try:    return series.astype(pd.Int32Dtype())
    except: pass
    try:    return series.astype(pd.Float32Dtype())
    except: pass
    try:    return series.astype(str).apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    except: pass
    try:
        if series.str.contains('\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}').all():
            return series.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        elif series.str.contains('\d{4}-\d{2}-\d{2}').all():
            return pd.to_datetime(series, format='%Y-%m-%d')
        elif series.str.isnumeric().all():
            return series.astype(pd.Int32Dtype())
        elif series.str.isnumeric().any():
            return series.astype(pd.Float32Dtype())
    except: pass
    return series 

def parse_html_table(html_text,**kwargs):
    return pd.concat(pd.read_html(html_text,flavor='bs4',**kwargs))

def parse_html_table_with_hrefs(html_text,columns=None):
    df_hrefs = parse_html_table(html_text,extract_links='all')
    # For each column in df, split the column into two columns, one for the text and one for the href
    org_columns = df_hrefs.columns 
    for col in df_hrefs.columns:
        col_name = col[1][0]
        df_hrefs[[col_name,col_name+'_href']] = pd.DataFrame(df_hrefs[col].tolist(), index=df_hrefs.index)
    # Drop the original columns and any empty columns
    df_hrefs = df_hrefs.drop(columns=org_columns)
    df_hrefs.dropna(axis=1,how='all',inplace=True)
    return df_hrefs

def __parse_boxscores_tables_type1__(html_text):
    html_soup = parse_html_soup(html_text)
    parsed_boxscores_tables = {}
    for table_elmt in html_soup.find_all('table'):
        table_id = table_elmt.get('id')
        if table_id in ['line_score','four_factors']:
            df = parse_html_table_with_hrefs(str(table_elmt))
            df.columns = df.columns.str.lower()
            df.rename({'':'team','_href':'team_href'},axis=1,inplace=True)
            parsed_boxscores_tables[table_id] = df
    return parsed_boxscores_tables


def __parse_boxscores_tables_type2__(html_text):
    # parse box scores that starts with box
    table_away_home_teams = __parse_game_info_tables__(html_text)['info-away-home-teams']
    away,home = table_away_home_teams['team_id']
    away_href,home_href = table_away_home_teams['team_href']

    html_soup = parse_html_soup(html_text)
    parsed_boxscores_tables = {}
    for table_elmt in html_soup.find_all('table'):
        table_id = table_elmt.get('id')
        if table_id and table_id.startswith('box'):
            if away in table_id:
                table_id = re.sub(away,'away',table_id)
                team = away
                team_href = away_href
            elif home in table_id:
                table_id = re.sub(home,'home',table_id)
                team = home
                team_href = home_href

            df_team_stats = parse_html_table_with_hrefs(table_elmt.prettify())
            df_team_stats.columns = df_team_stats.columns.str.lower()
            for na_keywords in ['Did Not Play','Did Not Dress','Coach\'s Decision','']:
                df_team_stats.replace(na_keywords, pd.NA,inplace=True)

            df_team_totals = df_team_stats[df_team_stats['starters'] == 'Team Totals'].copy()
            df_team_totals.rename(columns={'starters':'team','starters_href':'team_href'},inplace=True)
            df_team_totals[['team','team_href']] = [team,team_href]

            df_players_stats = df_team_stats[~df_team_stats['starters'].isin(('Reserves','Team Totals'))].copy()
            df_players_stats = df_players_stats.apply(convert_series_dtype)
            df_players_stats.rename(columns={'starters':'player','starters_href':'player_href'},inplace=True)
            df_players_stats.insert(0,'team',team)
            df_players_stats.insert(1,'team_href',team_href)
            df_players_stats['mp'] = df_players_stats['mp'].apply(convert_time_to_minutes)

            parsed_boxscores_tables[table_id] = df_players_stats
            parsed_boxscores_tables[table_id+'-total'] = df_team_totals
    return parsed_boxscores_tables

def __parse_game_info_tables__(html_text):
    html_soup = parse_html_soup(html_text)
    away,home = html_soup.select('div .scorebox strong a')
    df_game_info = []
    for team_type, team_element in zip(('away','home'),(away,home)):
        team_name = team_element.text
        team_href = team_element['href']
        team_id,season = re.search(r'/teams/(?P<team>\w+)/(?P<season>\d+)\.html',team_href).groups()
        df_game_info.append([team_id, team_href, team_name.strip(), season, team_type])
    df_game_info = pd.DataFrame(df_game_info,columns = ['team_id', 'team_href', 'team_name', 'season', 'team_type'])
    return {'info-away-home-teams': df_game_info}    
    
def parse_all_boxscores_tables(html_text):
    parsed_boxscores_tables = {
        **__parse_game_info_tables__(html_text),
        **__parse_boxscores_tables_type1__(html_text),
        **__parse_boxscores_tables_type2__(html_text),
    }
    return parsed_boxscores_tables

source = './data/boxscores/202110190LAL.html'
html_text = load_file(source)
html_text = re.sub("<!--|-->","\n",html_text)
parsed_boxscores_tables = parse_all_boxscores_tables(html_text)
# __parse_boxscores_tables_type2__(html_text)

iter_tables(parsed_boxscores_tables,1)

box-home-game-advanced-total: 100%|██████████| 35/35 [00:35<00:00,  1.01s/it]


In [114]:
source = './data/boxscores/202110190LAL.html'
html_text = load_file(source)
html_text = re.sub("<!--|-->","\n",html_text)

table_away_home_teams = __parse_game_info_tables__(html_text)['info-away-home-teams']
away,home = table_away_home_teams['team_id']
away_href,home_href = table_away_home_teams['team_href']

html_soup = parse_html_soup(html_text)
parsed_boxscores_tables = {}
for table_elmt in html_soup.find_all('table')[8:]:
    table_id = table_elmt.get('id')
    if table_id and table_id.startswith('box'):
        if away in table_id:
            table_id = re.sub(away,'away',table_id)
            team = away
            team_href = away_href
        elif home in table_id:
            table_id = re.sub(home,'home',table_id)
            team = home
            team_href = home_href

        df_team_stats = parse_html_table_with_hrefs(table_elmt.prettify())
        df_team_stats.columns = df_team_stats.columns.str.lower()
        for na_keywords in ['Did Not Play','Did Not Dress','Coach\'s Decision','']:
            df_team_stats.replace(na_keywords, pd.NA,inplace=True)

        df_team_totals = df_team_stats[df_team_stats['starters'] == 'Team Totals'].copy()
        df_team_totals.rename(columns={'starters':'team','starters_href':'team_href'},inplace=True)
        df_team_totals[['team','team_href']] = [team,team_href]

        df_players_stats = df_team_stats[~df_team_stats['starters'].isin(('Reserves','Team Totals'))].copy()
        df_players_stats = df_players_stats.apply(convert_series_dtype)
        df_players_stats.rename(columns={'starters':'player','starters_href':'player_href'},inplace=True)
        df_players_stats.insert(0,'team',team)
        df_players_stats.insert(1,'team_href',team_href)
        df_players_stats['mp'] = df_players_stats['mp'].apply(convert_time_to_minutes)

        parsed_boxscores_tables[table_id] = df_players_stats
        parsed_boxscores_tables[table_id+'-total'] = df_team_totals


In [8]:
# Save parsed boxscores tables as csv
parsed_dir = './data-parsed'
html_source = '/boxscores/202110190LAL.html'
html_text = load_file('./data' + html_source)
html_text = re.sub("<!--|-->","\n",html_text)
parsed_boxscores_tables = parse_all_boxscores_tables(html_text)
make_directory(f'{parsed_dir}{html_source.replace(".html","/")}')

tqdm_parsed_boxscores_tables = tqdm(parsed_boxscores_tables.items())
for table_id, df in tqdm_parsed_boxscores_tables:
    tqdm_parsed_boxscores_tables.set_description(table_id)
    df.to_csv(f'{parsed_dir}{html_source.replace(".html","/")}{table_id}.csv')



box-home-game-advanced-total: 100%|██████████| 35/35 [00:00<00:00, 862.30it/s]


In [138]:
# Parse gamelog tables
PARSED_DIR = './data-parsed'
html_source = './data/teams/ATL/2022/gamelog.html'
html_text = load_file(html_source)
html_text = clean_html_text(html_text)
html_soup = parse_html_soup(html_text)

html_basic_regular = html_soup.find('table', {'id': 'tgl_basic'})
html_basic_playoffs = html_soup.find('table', {'id': 'tgl_basic_playoffs'})

df_gamelog_basic = pt.parse_html_table(html_basic_regular.prettify())
df_gamelog_basic.rename(
    columns={'Unnamed: 3_level_1':'H/A', 
        **{f'Unnamed: {i}_level_0':'Match' for i in range(5)},
        **{f'Unnamed: {i}_level_0':'Result' for i in range(5,8)}},
    inplace=True)
df_gamelog_basic.drop(
    columns=[('Unnamed: 24_level_0', 'Unnamed: 24_level_1'),
             ('Match','Rk')], inplace=True)
game_filter = df_gamelog_basic.loc[:,('Match','G')].astype(str).str.isnumeric().fillna(False)
df_gamelog_basic = df_gamelog_basic.loc[game_filter,:]

df_gamelog_basic.loc[:,('Match','H/A')] = df_gamelog_basic[('Match','H/A')].isna().astype(int)
df_gamelog_basic.loc[:,('Result','W/L')] = df_gamelog_basic[('Result','W/L')].str.startswith('W').astype(int)
opponent_hrefs = pt.parse_html_table(html_basic_regular.prettify(),extract_links='body').loc[game_filter,'Unnamed: 4_level_0']
df_gamelog_basic.insert(4,('Match','Opp_href'),[href[0][-1] for href in opponent_hrefs.values])
df_gamelog_basic

Match                                            Result           Team       
       G        Date H/A  Opp              Opp_href    W/L   Tm  Opp   FG FGA   
0      1  2021-10-21   1  DAL  /teams/DAL/2022.html      1  113   87   45  94  \
1      2  2021-10-23   0  CLE  /teams/CLE/2022.html      0   95  101   38  99   
2      3  2021-10-25   1  DET  /teams/DET/2022.html      1  122  104   46  90   
3      4  2021-10-27   0  NOP  /teams/NOP/2022.html      1  102   99   40  96   
4      5  2021-10-28   0  WAS  /teams/WAS/2022.html      0  111  122   48  88   
..   ...         ...  ..  ...                   ...    ...  ...  ...  ...  ..   
83    78  2022-04-02   1  BRK  /teams/BRK/2022.html      1  122  115   36  86   
84    79  2022-04-05   0  TOR  /teams/TOR/2022.html      0  108  118   43  94   
85    80  2022-04-06   1  WAS  /teams/WAS/2022.html      1  118  103   43  92   
88    81  2022-04-08   0  MIA  /teams/MIA/2022.html      0  109  113   42  88   
89    82  2022-04-10   0  HOU  /teams/HOU/2022.html      1  130  114   45  87   

    ... Opponent                                        
    ...       FT FTA   FT% ORB TRB AST STL BLK TOV  PF  
0   ...       12  13  .923  10  50  16   7   3  15  21  
1   ...       20  27  .741  12  54  23   6   6  10  16  
2   ...       15  18  .833  11  36  26   7   6  14  15  
3   ...        8   9  .889   9  44  24   5   6   9  19  
4   ...       17  21  .810  14  51  26   6   4   6  16  
..  ...      ...  ..   ...  ..  ..  ..  ..  ..  ..  ..  
83  ...       14  19  .737  10  52  17   5   3  13  29  
84  ...       21  31  .677  20  60  20   6   5   7  14  
85  ...        9  11  .818   6  44  26   4   4  14  17  
88  ...       22  28  .786   4  33  20   9   1  17  17  
89  ...       15  20  .750   6  34  24   4   4   5  19  

[82 rows x 40 columns]

In [172]:
PARSED_DIR = './data-parsed'
team_href = '/teams/ATL/2022.html'
team = re.search(r'/teams/(?P<team>\w+)/(?P<season>\d+)\.html',team_href).groups()[0]
html_source = './data' + team_href.strip('.html') + '/gamelog-advanced.html'
html_text = load_file(html_source)
html_text = clean_html_text(html_text)

def __parse_team_gamelogs_basic__(html_text):
    df_gamelog_basic = pt.parse_html_table(html_text)
    # Rename columns
    df_gamelog_basic.rename(
        columns={'Unnamed: 3_level_1':'H/A', 
            **{f'Unnamed: {i}_level_0':'Match' for i in range(5)},
            **{f'Unnamed: {i}_level_0':'Result' for i in range(5,8)}},
        inplace=True)
    # Drop columns
    df_gamelog_basic.drop(
        columns=[('Unnamed: 24_level_0', 'Unnamed: 24_level_1'),
                ('Match','Rk')], inplace=True)
    # Filter out rows that are not games
    game_filter = df_gamelog_basic.loc[:,('Match','G')].astype(str).str.isnumeric().fillna(False)
    df_gamelog_basic = df_gamelog_basic.loc[game_filter,:]
    # Extract opponent hrefs
    opponent_hrefs = pt.parse_html_table(html_text,extract_links='body').loc[game_filter,'Unnamed: 4_level_0']
    df_gamelog_basic.insert(2,('Match','H/A'),df_gamelog_basic.pop(('Match','H/A')))
    df_gamelog_basic.insert(4,('Match','Opp_href'),[href[0][-1] for href in opponent_hrefs.values])
    # Convert columns to numeric
    df_gamelog_basic.loc[:,('Match','H/A')] = df_gamelog_basic[('Match','H/A')].isna().astype(int)
    df_gamelog_basic.loc[:,('Result','W/L')] = df_gamelog_basic[('Result','W/L')].str.startswith('W').astype(int)
    df_gamelog_basic = df_gamelog_basic.apply(pt.convert_series_dtype)
    df_gamelog_basic.reset_index(drop=True,inplace=True)

    return df_gamelog_basic

def __parse_team_gamelogs_advanced__(html_text):
    df_gamelog_advanced = pt.parse_html_table(html_text)
    # Rename columns
    df_gamelog_advanced.rename(
        columns={'Unnamed: 3_level_1':'H/A', 
            **{f'Unnamed: {i}_level_0':'Match' for i in range(5)},
            **{f'Unnamed: {i}_level_0':'Result' for i in range(5,8)}},
        inplace=True)
    # Drop columns
    df_gamelog_advanced.drop(
        columns=[('Unnamed: 23_level_0', 'Unnamed: 23_level_1'),
                ('Unnamed: 18_level_0', 'Unnamed: 18_level_1'),
                ('Match','Rk')], inplace=True)
    # Filter out rows that are not games
    game_filter = df_gamelog_advanced.loc[:,('Match','G')].astype(str).str.isnumeric().fillna(False)
    df_gamelog_advanced = df_gamelog_advanced.loc[game_filter,:]
    # Extract opponent hrefs
    opponent_hrefs = pt.parse_html_table(html_text,extract_links='body').loc[game_filter,'Unnamed: 4_level_0']
    df_gamelog_advanced.insert(2,('Match','H/A'),df_gamelog_advanced.pop(('Match','H/A')))
    df_gamelog_advanced.insert(4,('Match','Opp_href'),[href[0][-1] for href in opponent_hrefs.values])
    # Convert columns to numeric
    df_gamelog_advanced.loc[:,('Match','H/A')] = df_gamelog_advanced[('Match','H/A')].isna().astype(int)
    df_gamelog_advanced.loc[:,('Result','W/L')] = df_gamelog_advanced[('Result','W/L')].str.startswith('W').astype(int)
    df_gamelog_advanced = df_gamelog_advanced.apply(pt.convert_series_dtype)
    df_gamelog_advanced.reset_index(drop=True,inplace=True)
    return df_gamelog_advanced

def parse_all_team_gamelogs_tables(html_text):
    all_team_gamelogs_tables = {}
    for _id in ['tgl_basic','tgl_basic_playoffs','tgl_advanced', 'tgl_advanced_playoffs']:
        html_soup = parse_html_soup(html_text)
        tgl_table = html_soup.find('table', {'id': _id})
        if tgl_table:
            tgl_html = tgl_table.prettify()
            if _id in ['tgl_basic','tgl_basic_playoffs']:
                all_team_gamelogs_tables[_id] = __parse_team_gamelogs_basic__(tgl_html)
            elif _id in ['tgl_advanced', 'tgl_advanced_playoffs']:
                all_team_gamelogs_tables[_id] = __parse_team_gamelogs_advanced__(tgl_html)
    return all_team_gamelogs_tables

all_team_gamelogs_tables = parse_all_team_gamelogs_tables(html_text)
pt.iter_tables(all_team_gamelogs_tables,3)


tgl_advanced_playoffs: 100%|██████████| 2/2 [00:06<00:00,  3.02s/it]


In [5]:
PARSED_DIR = './data-parsed'
team_href = '/teams/ATL/2022.html'
team = re.search(r'/teams/(?P<team>\w+)/(?P<season>\d+)\.html',team_href).groups()[0]
html_source = './data-html' + team_href.strip('.html') + '/gamelog-advanced.html'
html_text = load_file(html_source)
html_text = clean_html_text(html_text)
html_soup = parse_html_soup(html_text)

html_advanced_regular = html_soup.find('table', {'id': 'tgl_advanced'})
html_advanced_playoffs = html_soup.find('table', {'id': 'tgl_advanced_playoffs'})

df_gamelog_advanced = pt.parse_html_table(html_advanced_regular.prettify())
# Rename columns
df_gamelog_advanced.rename(
    columns={'Unnamed: 3_level_1':'H/A', 
        **{f'Unnamed: {i}_level_0':'Match' for i in range(5)},
        **{f'Unnamed: {i}_level_0':'Result' for i in range(5,8)}},
    inplace=True)

# Drop columns
df_gamelog_advanced.drop(
    columns=[('Unnamed: 23_level_0', 'Unnamed: 23_level_1'),
             ('Unnamed: 18_level_0', 'Unnamed: 18_level_1'),
             ('Match','Rk')], inplace=True)

# Filter out rows that are not games
game_filter = df_gamelog_advanced.loc[:,('Match','G')].astype(str).str.isnumeric().fillna(False)
df_gamelog_advanced = df_gamelog_advanced.loc[game_filter,:]

# Extract opponent hrefs
opponent_hrefs = pt.parse_html_table(html_advanced_regular.prettify(),extract_links='body').loc[game_filter,'Unnamed: 4_level_0']
boxscores_hrefs = pt.parse_html_table(html_advanced_regular.prettify(),extract_links='body').loc[game_filter,'Unnamed: 2_level_0']
df_gamelog_advanced.insert(2,('Match','Boxscores_href'),[href[0][-1] for href in boxscores_hrefs.values])
df_gamelog_advanced.insert(3,('Match','H/A'),df_gamelog_advanced.pop(('Match','H/A')))
df_gamelog_advanced.insert(4,('Match','Tm'),team)
df_gamelog_advanced.insert(5,('Match','Tm_href'),team_href)
df_gamelog_advanced.insert(7,('Match','Opp_href'),[href[0][-1] for href in opponent_hrefs.values])


# Convert columns to numeric
df_gamelog_advanced.loc[:,('Match','H/A')] = df_gamelog_advanced[('Match','H/A')].isna().astype(int)
df_gamelog_advanced.loc[:,('Result','W/L')] = df_gamelog_advanced[('Result','W/L')].str.startswith('W').astype(int)
df_gamelog_advanced = df_gamelog_advanced.apply(pt.convert_series_dtype)
df_gamelog_advanced.reset_index(drop=True,inplace=True)
df_gamelog_advanced

Match                                                     
       G       Date                Boxscores_href H/A   Tm   
0      1 2021-10-21  /boxscores/202110210ATL.html   1  ATL  \
1      2 2021-10-23  /boxscores/202110230CLE.html   0  ATL   
2      3 2021-10-25  /boxscores/202110250ATL.html   1  ATL   
3      4 2021-10-27  /boxscores/202110270NOP.html   0  ATL   
4      5 2021-10-28  /boxscores/202110280WAS.html   0  ATL   
..   ...        ...                           ...  ..  ...   
77    78 2022-04-02  /boxscores/202204020ATL.html   1  ATL   
78    79 2022-04-05  /boxscores/202204050TOR.html   0  ATL   
79    80 2022-04-06  /boxscores/202204060ATL.html   1  ATL   
80    81 2022-04-08  /boxscores/202204080MIA.html   0  ATL   
81    82 2022-04-10  /boxscores/202204100HOU.html   0  ATL   

                                                    Result       ... Advanced   
                 Tm_href  Opp              Opp_href    W/L   Tm  ...     STL%   
0   /teams/ATL/2022.html  DAL  /teams/DAL/2022.html      1  113  ...      7.8  \
1   /teams/ATL/2022.html  CLE  /teams/CLE/2022.html      0   95  ...      5.2   
2   /teams/ATL/2022.html  DET  /teams/DET/2022.html      1  122  ...     11.1   
3   /teams/ATL/2022.html  NOP  /teams/NOP/2022.html      1  102  ...      4.3   
4   /teams/ATL/2022.html  WAS  /teams/WAS/2022.html      0  111  ...      4.0   
..                   ...  ...                   ...    ...  ...  ...      ...   
77  /teams/ATL/2022.html  BRK  /teams/BRK/2022.html      1  122  ...      9.5   
78  /teams/ATL/2022.html  TOR  /teams/TOR/2022.html      0  108  ...      5.2   
79  /teams/ATL/2022.html  WAS  /teams/WAS/2022.html      1  118  ...     11.3   
80  /teams/ATL/2022.html  MIA  /teams/MIA/2022.html      0  109  ...      9.4   
81  /teams/ATL/2022.html  HOU  /teams/HOU/2022.html      1  130  ...      4.2   

         Offensive Four Factors                           
    BLK%                   eFG%  TOV%       ORB% FT/FGA   
0   18.0                  0.559  11.7       13.0  0.085  \
1    4.9                  0.434   7.9  28.799999  0.091   
2    5.2                  0.578  11.6       28.6    0.2   
3    7.0                  0.458   9.6       37.5  0.146   
4    4.6                   0.58  12.1       14.0  0.102   
..   ...                    ...   ...        ...    ...   
77   6.5                  0.494   5.3  19.200001   0.43   
78   6.1                  0.532   6.6       20.0  0.085   
79   2.0                  0.554   5.6       15.6  0.174   
80   0.0                  0.523  12.7       31.0  0.193   
81   9.3                  0.638  11.0  31.700001  0.218   

   Defensive Four Factors                               
                     eFG%       TOV%       DRB% FT/FGA  
0                   0.403       13.2  83.099998  0.129  
1                   0.455        9.0       75.5  0.225  
2                   0.489       12.4       78.0  0.165  
3                   0.489        8.5  79.099998  0.086  
4                    0.53        5.3       72.5  0.172  
..                    ...        ...        ...    ...  
77                   0.51       10.8       81.5  0.141  
78                   0.48        5.8  62.299999  0.208  
79                  0.547       13.4  84.199997  0.105  
80                  0.632  16.799999  88.199997  0.306  
81                  0.556        4.9       86.0  0.169  

[82 rows x 29 columns]